# Problem 2: Analyzing the Netflix Data

In [1]:
# Change to the location of data files
dbfs_dir = 's3://archanamaroldsde.bucket/'
test = dbfs_dir + '/TestingRatings.txt'
train = dbfs_dir + '/TrainingRatings.txt'
names= dbfs_dir + '/movie_titles.txt'

In [2]:
test

's3://archanamaroldsde.bucket//TestingRatings.txt'

In [3]:
#MovieID,UserID,Rating

In [4]:
from pyspark.sql.functions import regexp_extract
from pyspark.sql.types import *
import pandas
tests=sqlContext.read.text(test)
trains=sqlContext.read.text(train)
movie_names=sqlContext.read.text(names)

In [5]:
import pyspark.sql.functions as f
test_data = tests.select(f.split(tests.value,",")).rdd.flatMap(lambda x: x).toDF(schema=["movieID","userID", "rating"])
train_data = trains.select(f.split(trains.value,",")).rdd.flatMap(lambda x: x).toDF(schema=["movieID","userID", "rating"])
movie_name = movie_names.select(f.split(movie_names.value,",")).rdd.flatMap(lambda x: x).toDF(schema=["movieID","year", "title"])


In [6]:
#converting types
test_data = test_data.withColumn("movieID",test_data["movieID"].cast(IntegerType()))
test_data = test_data.withColumn("userID",test_data["userID"].cast(IntegerType()))
test_data = test_data.withColumn("rating",test_data["rating"].cast(FloatType()))

train_data = train_data.withColumn("movieID",train_data["movieID"].cast(IntegerType()))
train_data = train_data.withColumn("userID",train_data["userID"].cast(IntegerType()))
train_data = train_data.withColumn("rating",train_data["rating"].cast(FloatType()))

movie_name = movie_name.withColumn("movieID",movie_name["movieID"].cast(IntegerType()))
movie_name = movie_name.withColumn("year",movie_name["year"].cast(IntegerType()))
movie_name = movie_name.withColumn("title",movie_name["title"].cast(StringType()))

In [7]:
print(test_data.show(3))
print(train_data.show(3))
print(movie_name.show(3))

+-------+-------+------+
|movieID| userID|rating|
+-------+-------+------+
|      8| 573364|   1.0|
|      8|2149668|   3.0|
|      8|1089184|   3.0|
+-------+-------+------+
only showing top 3 rows

None
+-------+-------+------+
|movieID| userID|rating|
+-------+-------+------+
|      8|1744889|   1.0|
|      8|1395430|   2.0|
|      8|1205593|   4.0|
+-------+-------+------+
only showing top 3 rows

None
+-------+----+--------------------+
|movieID|year|               title|
+-------+----+--------------------+
|      1|2003|     Dinosaur Planet|
|      2|2004|Isle of Man TT 20...|
|      3|1997|           Character|
+-------+----+--------------------+
only showing top 3 rows

None


In [8]:
test_data.cache()
train_data.cache()
assert test_data.is_cached
assert train_data.is_cached

In [9]:
train_data_count = train_data.count()
test_data_count = test_data.count()
print('There are %s rows in the train datasets' % (train_data_count))
print('There are %s rows in the test datasets' % (test_data_count))

There are 3255352 rows in the train datasets
There are 100478 rows in the test datasets


## Lets check if its workd for my own preferences

In [31]:
#Justice League=48, Dinosaur Planet=1, Horror Vision =41, Jade=55

 #### lets pick an user

In [54]:
x=[48,1,41,55]

In [55]:
userInput=spark.createDataFrame(
    [
        (1, 1,4.0), 
        (48,1,4.0), 
        (41,1, 4.0), 
        (55,1, 4.0),
    ],
    ['movieID', 'userID', 'rating'] 
)

In [56]:
userInput.show()
l = userInput.select('movieID')
a = [int(row.movieID) for row in l.collect()]

+-------+------+------+
|movieID|userID|rating|
+-------+------+------+
|      1|     1|   4.0|
|     48|     1|   4.0|
|     41|     1|   4.0|
|     55|     1|   4.0|
+-------+------+------+



 
 
 
 ####  The users who has seen the same movies from train dataset
 
 
 

In [82]:
similaruser = train_data[train_data['movieID'].isin(a)]
similaruser.show(5)
similaruser.count()

+-------+-------+------+
|movieID| userID|rating|
+-------+-------+------+
|     48|1567328|   3.0|
|     48|2445646|   3.0|
|     48|1395430|   2.0|
|     48|1174530|   3.0|
|     48| 542786|   3.0|
+-------+-------+------+
only showing top 5 rows



1666

We dont want to run for whole data so lets take only 100

In [60]:
similarusers=similaruser.select('userID').distinct()
similar=similarusers

In [61]:
#collecting in a list
l1 = similar.select('userID')
similaruser_ids = [int(row.userID) for row in l1.collect()]

#### Calculating the Pearson Correlation between input user and subset group, and store it in a dictionary

In [62]:
pearsonCorrelationDict={}
import  numpy as np
for i in similaruser_ids:
    #get each userid from the similarusers file and sort it by movieid
    group =  similaruser[similaruser['userID'].isin(i)]
    group=group.orderBy(group.movieID.desc())
    #group.show()
    #store them in an array 
    l2 = group.select('movieID')
    a2 = [int(row.movieID) for row in l2.collect()]
    #get each userInput  file and sort it by movieid
    userInput1=userInput.orderBy(userInput.movieID.desc())
    #userInput1.show()
    #get the number of rows of the groupfile
    nRatings = group.count()
    #Get the rating scores for the movies that they both have in common
    temp_df = userInput1[userInput1['movieID'].isin(a2)] 
    l3 = temp_df.select('rating')
    #store in a list
    rating_tempdf = [int(row.rating) for row in l3.collect()]
    #print("rating_tempdf",rating_tempdf)
    #Get the rating scores for the group data
    l4 = group.select('rating')
    #store in a list
    rating_group = [int(row.rating) for row in l4.collect()]
    #print("rating_group",rating_group)
    
    #Now let's calculate the pearson correlation between two users, so called, x and y
    Sxx = sum([i**2 for i in rating_tempdf]) - pow(sum(rating_tempdf),2)/float(nRatings)
    Syy = sum([i**2 for i in rating_group]) - pow(sum(rating_group),2)/float(nRatings)
    Sxy = sum( i*j for i, j in zip(rating_tempdf, rating_group)) - sum(rating_tempdf)*sum(rating_group)/float(nRatings)
    
    if Sxx != 0 and Syy != 0:
        pearsonCorrelationDict[i] = Sxy/(np.sqrt(Sxx*Syy))
    else:
        pearsonCorrelationDict[i] = 0 

In [63]:
pearsonCorrelationDict.items()

dict_items([(128389, 0), (455334, 0), (1628484, 0), (1909175, 0), (1849621, 0), (2232581, 0), (1559165, 0), (1215796, 0), (2520933, 0), (411705, 0), (41068, 0), (7576, 0), (2511987, 0), (1258560, 0), (814740, 0), (2615549, 0), (1349717, 0), (1436645, 0), (314043, 0), (767816, 0), (876961, 0), (952156, 0), (1877143, 0), (168108, 0), (821436, 0), (391543, 0), (2111014, 0), (2325737, 0), (49890, 0), (1145636, 0), (1339784, 0), (2012438, 0), (591794, 0), (2260788, 0), (2025375, 0), (2585641, 0), (2352463, 0), (2640278, 0), (1986937, 0), (1906611, 0), (298900, 0), (1573228, 0), (818752, 0), (1650301, 0), (1452675, 0), (2339734, 0), (1708366, 0), (599529, 0), (498716, 0), (864232, 0), (475873, 0), (929308, 0), (209024, 0), (688217, 0), (1746701, 0), (1259114, 0), (1932594, 0), (1640334, 0), (1191122, 0), (2426088, 0), (139433, 0), (2173428, 0), (267231, 0), (384710, 0), (1061195, 0), (92056, 0), (49585, 0), (1743381, 0), (2601274, 0), (1005185, 0), (2151468, 0), (514829, 0), (1899913, 0), (8

#### Converting dictionary to dataframe and shorting in descending order


In [64]:
DF = pandas.DataFrame.from_dict(pearsonCorrelationDict, orient='index')
DF.columns = ['similarityIndex']
DF['userId'] = DF.index
DF.index = range(len(DF))
ddf = spark.createDataFrame(DF)

In [65]:
ddf.show()

+---------------+-------+
|similarityIndex| userId|
+---------------+-------+
|              0| 128389|
|              0| 455334|
|              0|1628484|
|              0|1909175|
|              0|1849621|
|              0|2232581|
|              0|1559165|
|              0|1215796|
|              0|2520933|
|              0| 411705|
|              0|  41068|
|              0|   7576|
|              0|2511987|
|              0|1258560|
|              0| 814740|
|              0|2615549|
|              0|1349717|
|              0|1436645|
|              0| 314043|
|              0| 767816|
+---------------+-------+
only showing top 20 rows



#### The top x similar users to input user

In [66]:
topUsers=ddf.orderBy(ddf.similarityIndex.desc())

In [67]:
topUsers.show()

+---------------+-------+
|similarityIndex| userId|
+---------------+-------+
|              0|2234535|
|              0| 719755|
|              0|1079032|
|              0|1129423|
|              0|1589788|
|              0| 378348|
|              0| 301501|
|              0| 236564|
|              0|1817718|
|              0|2357895|
|              0| 589124|
|              0|2559537|
|              0|  26210|
|              0|1172958|
|              0|1583664|
|              0| 257095|
|              0|1711710|
|              0|1572469|
|              0|2579439|
|              0| 230966|
+---------------+-------+
only showing top 20 rows



## Now, recommending movies to the input user.

#### Taking the weighted average of the ratings of the movies using the Pearson Correlation as the weight.
But to do this, we first need to get the movies watched by the users in our __DF__ from the ratings dataframe and then store their correlation in a new column called _similarityIndex". This is achieved below by merging of these two tables.

In [68]:
new_df = topUsers.join(similaruser, on=['userId' ], how='inner')

In [69]:
new_df.show()

+-------+---------------+-------+------+
| userId|similarityIndex|movieID|rating|
+-------+---------------+-------+------+
|2580300|              0|     48|   3.0|
| 901520|              0|     48|   3.0|
|1143750|              0|     48|   4.0|
|1993796|              0|     48|   3.0|
|2148401|              0|     48|   5.0|
|1241893|              0|     48|   5.0|
|1568735|              0|     48|   5.0|
|1685548|              0|     48|   3.0|
| 667102|              0|     48|   4.0|
| 522221|              0|     48|   4.0|
|2524282|              0|     48|   4.0|
|1346326|              0|     48|   5.0|
| 753107|              0|     48|   4.0|
|2154672|              0|     48|   4.0|
|2571687|              0|     48|   4.0|
| 904842|              0|     48|   4.0|
|1722278|              0|     48|   5.0|
|1058639|              0|     48|   4.0|
|1249577|              0|     48|   4.0|
|1664010|              0|     48|   5.0|
+-------+---------------+-------+------+
only showing top

Lets calculate the similarity index by multiply the movie rating by its weight, 
then sum up the new ratings and divide it by the sum of the weights.

It shows the idea of all similar users to candidate movies for the input user:

In [70]:
from pyspark.sql.functions import col

In [71]:
#multiply
new_df1=new_df.withColumn('weightedRating', (col('similarityIndex') * col('rating')))
new_df1.show()

+-------+---------------+-------+------+--------------+
| userId|similarityIndex|movieID|rating|weightedRating|
+-------+---------------+-------+------+--------------+
|2580300|              0|     48|   3.0|           0.0|
| 901520|              0|     48|   3.0|           0.0|
|1143750|              0|     48|   4.0|           0.0|
|1993796|              0|     48|   3.0|           0.0|
|2148401|              0|     48|   5.0|           0.0|
|1241893|              0|     48|   5.0|           0.0|
|1568735|              0|     48|   5.0|           0.0|
|1685548|              0|     48|   3.0|           0.0|
| 667102|              0|     48|   4.0|           0.0|
| 522221|              0|     48|   4.0|           0.0|
|2524282|              0|     48|   4.0|           0.0|
|1346326|              0|     48|   5.0|           0.0|
| 753107|              0|     48|   4.0|           0.0|
|2154672|              0|     48|   4.0|           0.0|
|2571687|              0|     48|   4.0|        

In [72]:
from pyspark.sql import functions as F
temp=new_df1.groupBy('movieID').agg(F.sum(new_df1.similarityIndex).alias("SI_sum"),F.sum(new_df1.weightedRating).alias("WR_sum"))

In [73]:
temp.show()

+-------+------+------+
|movieID|SI_sum|WR_sum|
+-------+------+------+
|     48|     0|   0.0|
+-------+------+------+



#### weighted average

In [74]:
recommendation=temp.withColumn('weighted_average', (col('WR_sum') / col('SI_sum')))
recommendation=recommendation.select("movieID","weighted_average")
recommendation=recommendation.orderBy(recommendation.weighted_average.desc()).limit(10)


#### These are the movies that  are the predicted rating for my preference 

#### Predicted

In [76]:
recommendation.show(5)

+-------+----------------+
|movieID|weighted_average|
+-------+----------------+
|     48|            null|
+-------+----------------+



In [77]:
m = recommendation.select('movieID')
n = [int(row.movieID) for row in m.collect()]

#### Actual

In [78]:
actual=userInput[userInput['movieID'].isin(n)]

In [79]:
actual.show()

+-------+------+------+
|movieID|userID|rating|
+-------+------+------+
|     48|     1|   4.0|
+-------+------+------+



In [ ]:
#Extracting the movie names 

In [ ]:
final1 = userInput.join(recommendation, on=['movieID' ], how='inner')
final2= final1.join(movie_name, on=['movieID' ], how='inner')